In [ ]:
import requests
import time
import random
import pandas as pd
import re
from bs4 import BeautifulSoup

x_values = ['a10a3f92e9--item--hwrM1', 'a10a3f92e9--item--iWTsg', 'a10a3f92e9--item--qJhdR']
columns_to_convert_bin = ['Холодильник', 'Посудомоечная машина', 'Стиральная машина', 'Кондиционер', 'Телевизор', 'Интернет', 'Мебель на кухне', 'Мебель в комнатах']
columns = ['Холодильник', 'Посудомоечная машина', 'Стиральная машина', 'Кондиционер', 'Телевизор', 'Интернет',
           'Мебель на кухне', 'Мебель в комнатах', 'Оплата ЖКХ', 'Залог', 'Комиссии', 'Предоплата', 'Срок аренды',
           'Условия проживания', 'Общая площадь', 'Жилая площадь', 'Площадь кухни', 'Санузел', 'Балкон/лоджия',
           'Вид из окон', 'Ремонт', 'Год постройки', 'Количество лифтов', 'Тип перекрытий', 'Парковка',
           'Отопление', 'Аварийность', 'link'] # как-то повелся такой порядок, потом нужно просто поставить link в начало
urls = pd.read_csv('файл_со_ссылками.csv')['add_link'][:] # слайсы ссылок

def convert_to_binary(value):
    return 1 if value else 0

def remove_column_names_from_values(df, column_names):
    for column_name in column_names:
        if column_name in df.columns:
            df[column_name] = df[column_name].apply(lambda x: x.replace(column_name, '').strip() if isinstance(x, str) else x)
    return df

def process_links(urls, x_values, columns, columns_to_convert_bin, save_interval=20): # основная функция
    all_data = []
    error_list = []
    
    for index, url in enumerate(urls):
        try:
            
            response = requests.get(url) # Получение HTML
            time.sleep(random.uniform(18, 23))# Здесь и далее подобраны значения для минимального количества ошибок от количества запросов
            response.raise_for_status()
            html = response.text
            
            
            soup = BeautifulSoup(html, 'html.parser') # Извлечение характеристик
            features = []
            for x in x_values:
                for i in soup.find_all('div', class_=x):
                    features.append(i.text.strip())
            
            
            row = {column: None for column in columns}
            for feature in features: # Распаковка данных
                for column in columns:
                    if column in feature:
                        extracted_value = re.sub(rf'{column}.*?(\d+)', r'\1', feature)
                        row[column] = extracted_value.strip()
                        break
            row['url'] = url
            all_data.append(row)
            
            print(f'Обработано: {index+1} {url}')
            
            
            if (index + 1) % save_interval == 0:# Сохранение каждые save_interval ссылок
                save_data(all_data, error_list, columns_to_convert_bin)
                all_data.clear()
                time.sleep(random.uniform(60, 100))
            if (index + 1) % 35 == 0:
                time.sleep(random.uniform(300, 600))
                
        except requests.exceptions.RequestException as e:
            print(f'Ошибка при обработке {index+1} {url}: {e}')
            error_list.append({'url': url, 'error': str(e)})
    
    if all_data:
        save_data(all_data, error_list, columns_to_convert_bin)

def save_data(data, errors, columns_to_convert_bin):
    df = pd.DataFrame(data)
    
    for column in columns_to_convert_bin:
        df[column] = df[column].apply(convert_to_binary)

    text_columns = [col for col in df.columns if col not in columns_to_convert_bin + ['url']]
    df = remove_column_names_from_values(df, text_columns)
    
    df.to_csv('processed_data.csv', mode='a', index=False, header=not bool(pd.read_csv('processed_data.csv').shape[0]))
    
    if errors:
        df_errors = pd.DataFrame(errors)
        df_errors.to_csv('errors.csv', mode='a', index=False, header=not bool(pd.read_csv('errors.csv').shape[0]))

In [ ]:
process_links(urls, x_values, columns, columns_to_convert_bin, save_interval=10)